In [1]:
!pip install fuzzymatcher

In [2]:
import pandas as pd
from pathlib import Path
import fuzzymatcher

In [3]:
folder_path = '/Users/victorweber/Desktop/Adressabgleich_2022/'
ofrex = pd.read_excel(f'{folder_path}ofrex.xlsx')
afa = pd.read_excel(f'{folder_path}afa.xlsx')

In [4]:
ofrex['match_1'] = (ofrex['Name (W1)'] + ofrex['Strasse (W4)'] + ofrex['PLZ'] + ofrex['Ort (W5)']).astype(str).map(lambda x: x.lower())
ofrex['match_2'] = (ofrex['Name (W1)']).astype(str).map(lambda x: x.lower())
afa['match_1'] = (afa['Nom 1'] + afa['Adresse'] + afa['Format adresse']).astype(str).map(lambda x: x.lower())
afa['match_2'] = (afa['Nom 1']).astype(str).map(lambda x: x.lower())


In [5]:
#Think before running! (>6 mins for 50kx2k matrices)
left_on = ['match_1']
right_on = ['match_1']
matched_results = fuzzymatcher.fuzzy_left_join(ofrex, afa, left_on, right_on)

In [29]:
matched_results = matched_results.sort_values(by = 'best_match_score', ascending = False)
matched_results_cleaned = matched_results[['best_match_score', 'AdrNr.', 'Name (W1)', 'Strasse (W4)', 'PLZ', 'Ort (W5)', 'Nom 1', 'Adresse', 'Format adresse', "Chiffre d'affaire (a.p.)"]]
matched_results_cleaned = matched_results_cleaned[(matched_results_cleaned.best_match_score >= 0.67)]

In [8]:
#Think before running! (>2 mins for 50kx2k matrices)
left_on = ['match_2']
right_on = ['match_2']
matched_results_2 = fuzzymatcher.fuzzy_left_join(ofrex, afa, left_on, right_on)

In [32]:

matched_results_2.sort_values(by = 'best_match_score', ascending = False)
matched_results_cleaned_2 = matched_results_2[['best_match_score', 'AdrNr.', 'Name (W1)', 'Strasse (W4)', 'PLZ', 'Ort (W5)', 'Nom 1', 'Adresse', 'Format adresse', "Chiffre d'affaire (a.p.)"]]
matched_results_cleaned_2 = matched_results_cleaned_2.sort_values(by = 'best_match_score', ascending = False)
matched_results_cleaned_2 = matched_results_cleaned_2[(matched_results_cleaned_2.best_match_score >= 0.42)]
matched_results_appended = matched_results_cleaned.append(matched_results_cleaned_2)
matched_results_appended = matched_results_appended.drop_duplicates(subset = ['AdrNr.'])

In [33]:
matched_results_appended.to_excel(f'{folder_path}ofrex_afa_match.xlsx', engine='xlsxwriter')

221